<a href="https://colab.research.google.com/github/wagnermcosta/bocth/blob/main/bocth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Estudo Bag of Concepts - Thesaurus

> Bloco com recuo



https://github.com/hank110/bagofconcepts  
Wagner Miranda Costa dez/2023

In [ ]:
!pip install rank_bm25
!pip install bagofconcepts
!pip install bottleneck

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

#descomente linha abaixo para execução no Google Colab
files_path = "/content/drive/My Drive/dados/"

#descomente linha abaixo para execução local
#files_path = "./"

In [ ]:
#configuraçao default para word2vec
embedding_dim = 100
context = 8
min_freq = 30
iterations = 5
concepts = 100

In [ ]:
!python --version

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


import matplotlib.pyplot as plt

import time

%matplotlib inline

***

In [ ]:
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
# Import and download stopwords from NLTK.
from nltk.corpus import stopwords
from nltk import download
import nltk
from gensim.similarities import SparseTermSimilarityMatrix, WordEmbeddingSimilarityIndex

# Download stopwords list
download('stopwords')
stop_words = stopwords.words('portuguese')
download('wordnet')
download('omw-1.4')

In [ ]:
import spacy

#lemmatizador em portugues
#descomentar as linhas abaixo no Google Colab quando da primeira compilação (ou se der erro na carga da biblioteca pt_core_news_lg)
import spacy.cli
spacy.cli.download("pt_core_news_lg")
import pt_core_news_lg
nlp = pt_core_news_lg.load()

#descomentar linha abaixo no jupyter
#nlp = spacy.load('pt_core_news_lg')

In [ ]:
#lemmatiza um texto
#sentence: texto no formato string ou list
#retorna: list de lemmas da sentence
def lemma(sentence):
    #verifica se argumento é list
    if type(sentence) is list:
        sentence = ' '.join(sentence)

    doc = nlp(sentence)
    #o "if not ' ' in token.lemma_" desconsidera lemma como 'em o' (na), 'por o' (pelo), etc.
    return [token.lemma_.lower() for token in doc if (not ' ' in token.lemma_ and len(token.text) > 2) ]
    #a linha abaixo deve ser descomentada, em substituição aa anterior, quando não se quiser aplicar lemmatização
    #return [token.text.lower() for token in doc if (not ' ' in token.text and len(token.text) > 2) ]



def isVerb(word):
    token = nlp(word)[0]
    return token.pos_ == "VERB"


def isNoun(word):
    token = nlp(word)[0]
    return token.pos_ == "NOUN"

def getPartOfSpeech(word):
    token = nlp(word)[0]
    return token.pos_




In [ ]:
%%time
#monta lista de Vocabulario de Controle Externo (VCE)
#corresponde à coluna termo da planilha de VCE, sem espaços em branco e outros caracteres

from re import sub

#converte para minúsculo, remove acentos e números
def prepareString(termo):
    termo = termo.replace('(',' ')
    termo = termo.replace(')',' ')
    termo = termo.replace('-',' ')
    termo = termo.replace(',',' ')
    termo = termo.replace('/',' ')
    termo = termo.replace('.',' ')
    termo = termo.replace(':',' ')
    termo = termo.replace(';',' ')
    termo = termo.replace('$','')
    termo = sub(r'[0-9]', " ", termo)
    return termo.lower()


#lematiza VCE
thesaurus_words = []
temp = pd.read_excel(files_path + 'vce.xlsx').termo.map(prepareString)
for line in temp:
    for word in line.split():
        thesaurus_words.append(word)

thesaurus_words = list(set(thesaurus_words))

#thesaurus_words_lemma corresponde ao tesauro de controle externo
thesaurus_words_lemma = lemma(thesaurus_words)
thesaurus_words_lemma = list(set(thesaurus_words_lemma))

translation = {39: None}
thesaurus_words_lemma = [token for token in thesaurus_words_lemma]
thesaurus_words_lemma.append('certame')

In [ ]:
#verificação da similaridade
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from re import sub
from gensim.utils import simple_preprocess
from scipy.spatial.distance import cityblock



#FUNÇÕES DE CÁLCULO DE SIMILARIDADE E DE PREPROCESSAMENTO


#preprocessa um texto
#argumento: texto no formato string
#retorna: lista de tokens
def preprocess(sentence):
    # Tokenize, clean up input document string
    sentence = sub(r'<img[^<>]+(>|$)', " ", sentence)
    sentence = sub(r'<[^<>]+(>|$)', " ", sentence)
    sentence = sub(r'\[img_assist[^]]*?\]', " ", sentence)
    sentence = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " ", sentence)
    sentence = prepareString(sentence)
    #remove STOPWORDS (realizado preliminarmente para não comprementer lexico da lemmatização com artigos e preposições)
    sentence = [token for token in sentence.lower().split() if token not in stop_words]

    #verifica novamente não pertencer aos STOPWORDS
    return [token for token in lemma(sentence) if token not in stop_words]


#preprocessa um texto, desprezando palavras que não constem do Tesauro
#argumento: texto no formato string
#retorna: lista de tokens
def preprocessOnlyThesaurus(sentence):

    #considera apenas termos do thesaurus
    return [token for token in preprocess(sentence) if (token in thesaurus_words_lemma)]



#vetorizador usando preprocessamento
vectorizer = CountVectorizer(tokenizer=preprocess)

#vetorizador TF-IDF usando preprocessamento
tfidfvec = TfidfVectorizer(tokenizer=preprocess)



def similarity(model, text_sentence1, text_sentence2, w2v_sentence1, w2v_sentence2):

    documents = [text_sentence1, text_sentence2]
    print('documents', documents)

    dictionary = Dictionary(documents)
    print(dictionary)

    termsim_index = WordEmbeddingSimilarityIndex(model)
    termsim_matrix = SparseTermSimilarityMatrix(termsim_index, dictionary)
    return termsim_matrix.inner_product(w2v_sentence1, w2v_sentence2, normalized=(True, True))



#calculo de similaridade semantica usando Soft Cosine Measure (SCM)
def similaritySCM(model, sentence1, sentence2):

     documents = [sentence1, sentence2]
     dictionary = Dictionary(documents)


     #sentence1 = dictionary.doc2bow(sentence1)
     #sentence2 = dictionary.doc2bow(sentence2)

     documents = [sentence1, sentence2]
     tfidf = TfidfModel(documents)

     sentence1 = tfidf[sentence1]
     sentence2 = tfidf[sentence2]

     print('***')
     print(sentence1)
     print(sentence2)

     termsim_index = WordEmbeddingSimilarityIndex(model)

     termsim_matrix = SparseTermSimilarityMatrix(termsim_index, dictionary, tfidf)

     return termsim_matrix.inner_product(sentence1, sentence2, normalized=(True, True))


#calculo da distancia WMD
def wmDistance(sentence1, sentence2):

    return 0
    # return model.wmdistance(sentence1, sentence2)


#calculo similaridade bag of words
def similarityBOW(sentence1, sentence2):

    tfidf = tfidfvec.fit_transform([sentence1, sentence2])
    return 1 - (((tfidf * tfidf.T).A)[0,1])

#similaridade jaccard binária
def jaccard_binary(x,y):
    """A function for finding the similarity between two binary vectors"""
    intersection = np.logical_and(x, y)
    union = np.logical_or(x, y)
    similarity = intersection.sum() / float(union.sum())
    return similarity

#similaridade jaccard conjunto
def jaccard_set(list1, list2):
    """Define Jaccard Similarity function for two sets"""
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union



#Similaridade cosseno
#recebe como parametros os vetores de caracteristicas já normalizados
#ou duas strings representando as sentenças a serem comparadas. Nesse caso, é calculado o TF-IDF das sentenças,
#e os vetores resultantes são normalizados
def cosineSimilarity (arg1, arg2):
    '''
    #verifica se parametros são strings (na verdade, verifica apenas o primeiro argumento!)
    if isinstance(arg1, str):
        #cria bow baseado em TF-IDF
        bow = tfidfvec.fit_transform([arg1, arg2])
        #bow = vectorizer.fit_transform([arg1, arg2])

        #normaliza os vetores baseado na soma do proprio array
        arg1 = bow.A[0]/np.sum(bow.A[0])
        arg2 = bow.A[1]/np.sum(bow.A[1])
    '''
    return np.dot(arg1, arg2)/(np.linalg.norm(arg1)*np.linalg.norm(arg2))

#Cosine distance
# Equivale a 1 - similaridade por cosseno
#recebe como parametros os vetores de caracteristicas já normalizados
#ou duas strings representando as sentenças a serem comparadas. Nesse caso, é calculado o TF-IDF das sentenças,
#e os vetores resultantes são normalizados
def cosineDistance (arg1, arg2):

    return 1 - cosineSimilarity(arg1, arg2)


#calculo similaridade bag of concepts
def euclideanDistance(sentence1, sentence2):
    temp = sentence1 - sentence2
    return np.sqrt(np.dot(temp.T, temp))



#calculo da distância Manhattan (cityblock)
def manhattanDistance (arg1, arg2):
    return cityblock(arg1, arg2)




def glaucoDistance(arg1, arg2):

    x = np.copy(arg1)
    y = np.copy(arg2)

    #normaliza vetores
    #a normalização deve ser a primeira operação, para considerar no max() os elementos que eventualmente
    #serão "zerados" na operação seguinte
    #x = x/[np.max(x)]
    #y = y/[np.max(y)]

    #"zera" elementos onde seu equivalente no outro vetor é igual a zero
    #desta forma, a diferença elementWise dessas coordenadas será zero
    x[np.where(y == 0)]  = 0
    y[np.where(x == 0)]  = 0

    #calcula manhattanDistance
    return cityblock(x ,y)


def glaucoDistanceOriginal(x,y):
  soma = 0
  refX = max(x)
  refY = max(y)
  for i in range(0, len(x)):
      if(y[i] > 0 and x[i] > 0):
          soma += abs( float((x[i]/refX)) - float((y[i]/refY)) )

  return soma



#Euclidean distance usando linalg.norm()
#recebe como parametros os vetores de caracteristicas já normalizados
#ou duas strings representando as sentenças a serem comparadas. Nesse caso, é calculado o TF-IDF das sentenças,
#e os vetores resultantes são normalizados
def euclidean (arg1, arg2):

    #verifica se parametros são strings (na verdade, verifica apenas o primeiro argumento!)
    if isinstance(arg1, str):
        #cria bow baseado em TF-IDF
        bow = tfidfvec.fit_transform([arg1, arg2])
        #bow = vectorizer.fit_transform([arg1, arg2])

        #normaliza os vetores baseado na soma do proprio array
        arg1 = bow.A[0]/np.sum(bow.A[0])
        arg2 = bow.A[1]/np.sum(bow.A[1])

    #Euclidean distance usando linalg.norm()
    return np.linalg.norm(arg1 - arg2)

---

## Extendendo classe BOCModel (bag of concepts)

In [ ]:
import bagofconcepts as boc
import pickle
from collections import Counter
from sklearn.utils.extmath import safe_sparse_dot
import scipy.sparse
from soyclustering import SphericalKMeans
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
import numpy as np
#import umap

# classe BOCModel2
# inherits from BOCModel para permitir calculo de similaridade
# com matriz bag of concepts gerada a partir do corpus de treinamento (calculos estao na classe BOCModel)
class BOCModel2(boc.BOCModel):

    #armazenara concepts-frequencies calculado em _apply_cfidf
    #para poder ser reutilizado em _get_cfidf
    cf = None
    #cf de cada documento do corpus ponderado pela soma de cf do proprio documento
    wcfdoc = None

    #valores utilizados para geração do word2vec
    num_embedding_dim = 0
    num_context = 0

    #classe metodo de clusterizacao utilizado
    cluster_class = None

    #culsterizacoes disponíveis
    clusterMethodAllowed = ['km','skm']
    #metodo utilizado para clusterizacao
    clusterMethod = None

    #indica se o modelo de clusterização será lido de arquivo gravado anteriormente
    loadCluster = False;

    #arquivo onde será armazenado o cluster
    clusterFile = files_path + 'boc2cluster.pickle'

    #construtor sobrescrito para receber valor para variáveis de geração do word2vec
    #
    #clusterMeth {'km', skm'}: tipo de clusterizaçao
    def __init__(self, corpus, wv, idx2word, num_concept=100, iterations=5, random_state=42,
                 num_embedding_dim=100, num_context=8, clusterMethod ='skm', loadCluster = False):
        super().__init__(corpus, wv, idx2word, num_concept, iterations, random_state)

        self.num_embedding_dim = num_embedding_dim
        self.num_context = num_context
        self.loadCluster = loadCluster

        #verifica se tipo de clusterização é válida
        if clusterMethod in self.clusterMethodAllowed:
            self.clusterMethod = clusterMethod
        else:
            raise ValueError('Método de clusterização %s inválido. Métodos disponíveis: %s.' % (clusterMethod, self.clusterMethodAllowed))




    #reaporoveita metodo BOCModel._create_bow(self):
    #porem retornando o resultado ao inves de atribuir aa variavel da classe
    def _get_bow(self, document, _idx2word=None):
        rows=[]
        cols=[]
        vals=[]
        word2idx={word:idx for idx, word in enumerate(self.idx2word)}

        for i, doc in enumerate(document):
            tokens_count=Counter([word2idx[token] for token in doc if token in word2idx])
            for idx, count in tokens_count.items():
                rows.append(i)
                cols.append(idx)
                vals.append(float(count))

        return csr_matrix((vals, (rows, cols)), shape=(i+1, len(word2idx)))


    # retorna vetor de características de um único documento, baseado
    # no BagOfConcepts do corpus de treinamento
    # document = documento pre-processado no formato list []
    def getwcf(self, document, exibeMatrix = False):

        #recupera w2v e indice, mas apenas o indice interessa
        #_, _idx2word = boc.utils.train_gensim_w2v(corpus=document, embedding_dim=self.num_embedding_dim,
        #                                          context=self.num_context, min_freq=1, iterations=self.iterations)
        #cria BoW
        #linha abaixo comentado pois o parâmetro _idx2word não é utilizado. Em seu lugar, o metodo
        #self._get_bow() utiliza variavel de classe que contem este  mesmo conteudo
        #_bow = self._get_bow(document, _idx2word)
        _bow = self._get_bow(document)


        csr_matrix = safe_sparse_dot(_bow, self.w2c)

        if exibeMatrix:
            print('csr_matrix', document, csr_matrix.get_shape(), csr_matrix.toarray() )

        return csr_matrix.toarray()[0]

    #reaproveita metodo BOCModel._apply_cfidf(self, csr_matrix):
    #porem retornando o resultado ao inves de atribuir aa variavel da classe
    def _get_cfidf(self, csr_matrix):

        num_docs, num_concepts = csr_matrix.shape

        trace('csr_matrix ')
        trace(csr_matrix)
        trace('num_docs ' + str(num_docs))
        trace('num_concepts ' +str(num_concepts))

        _, nz_concept_idx = csr_matrix.nonzero()

        trace('len(nz_concept_idx): %d', str(len(nz_concept_idx)))
        trace('nz_concept_idx ')
        trace(nz_concept_idx)

        cf = np.bincount(nz_concept_idx, minlength=num_concepts)

        #combinacao com o cf da classe (baseado no corpus original)
        newcf = np.multiply(cf, self.cf)


        #codigo original comentado para utilizar
        #a quantidade de documentos do corpus original
        #icf = np.log(num_docs / cf)
        icf = np.log(len(self.corpus) / newcf)


        icf[np.isinf(icf)] = 0


        trace('novo icf')
        trace(icf)

        trace('scipy.sparse.diags(icf)')
        trace(scipy.sparse.diags(icf))

        return safe_sparse_dot(csr_matrix, scipy.sparse.diags(icf))


    #metodo reproduzido na subclasse para poder armazenar o concepts-frequencies (cf) como
    #variavel de classe
    def _apply_cfidf(self, csr_matrix):
        num_docs, num_concepts = csr_matrix.shape
        _, nz_concept_idx = csr_matrix.nonzero()

        #codigos comentados para armazenar variavel de classe self.cf
        #cf = np.bincount(nz_concept_idx, minlength=num_concepts)
        #icf = np.log(num_docs / cf)
        self.cf = np.bincount(nz_concept_idx, minlength=num_concepts)
        icf = np.log(num_docs / self.cf)

        #calcula cf ponderado para cada doc do corpus
        #baseado no cf do próprio documento
        self.wcfdoc = []
        csr_matrix_temp = csr_matrix.toarray()
        for i in range(num_docs):
            self.wcfdoc.append(np.array(csr_matrix_temp[i]/np.sum(csr_matrix_temp[i])))

        icf[np.isinf(icf)] = 0

        self.boc = safe_sparse_dot(csr_matrix, scipy.sparse.diags(icf))

    #retorna array de concepts frequency (ou relativo a um documento do corpus de treinamento, caso index informado)
    def getwcfdoc(self, index=None):
        if (index == None):
            return self.boc.A
        else:
            return self.boc[index].A[0]


    #metodo reproduzido na subclasse para armazenar skm (SphericalKMeans)
    def _cluster_wv(self, wv, num_concept, max_iter=10):

        #verifica se le cluster gerado anteriormente
        if self.loadCluster:

            #lê cluster gerado e gravado anteriormente
            dbfile = open(self.clusterFile, 'rb')
            self.cluster_class = pickle.load(dbfile)
            dbfile.close()

            #atribui "labels_" ao atributo self.wv_cluster_id
            self.wv_cluster_id = self.cluster_class.labels_

        else :

            #não lê cluster gerado anteriormente: efetua nova clusterização
            sM=scipy.sparse.csr_matrix(wv).astype('double')


            #seleciona tipo de clusterizacao
            if self.clusterMethod == 'skm':
                self.cluster_class = SphericalKMeans(n_clusters=num_concept, max_iter=max_iter, verbose=0, init='similar_cut', sparsity='None')
            elif self.clusterMethod == 'km':
                self.cluster_class = KMeans(n_clusters=num_concept,  n_init=10, max_iter=max_iter, init='k-means++')
            else:
                raise Error('Método de clusterização inválido: %s. Válido: %s' % (self.clusterMethod,self.clusterMethodAllowed))


            #clusteriza e grava resultado ("labels_") ao atributo self.wv_cluster_id
            self.wv_cluster_id = self.cluster_class.fit_predict(sM)

            #grava cluster para eventual leitura posterior
            dbfile = open(self.clusterFile, 'ab')
            # source, destination
            pickle.dump(self.cluster_class, dbfile)
            dbfile.close()





    #retorna classe de clusterização
    def getClusterClass(self):
        return self.cluster_class

    #retorna mapeamento word -> concept no formato Dictionary
    def getDictWord2Concept(self):
        return dict([wc_pair for wc_pair in zip(self.idx2word, self.wv_cluster_id)])




In [ ]:
#classe para identificar (ou predizer) o conceito definido em um modelo bag-of-concepts (BOC)

import scipy.spatial.distance as sdist
import networkx as nx
import bottleneck as bn


# classe conceptCluster
class conceptCluster:

    #modelo que possui os clusters de conceitos
    cluster_model = None

    #word2vec que será utilizado nas predições
    w2v = None

    #dicionario que mapeia cada palavra ao seu conceito
    word2concept = None

    #grafo de relacionamento dos conceitos
    conceptGraph = None

    #distancia (no grafo) entre os conceitos
    #a chave é a tupla (conceito1, conceito2)
    conceptDistance = {}

    #construtor
    #cluster_model: modelo com os clusters de conceitos
    #word2vec: word2vec utilizado para a geração dos conceitos
    #word2concept: dicionario que mapeia cada palavra utilizada para geração do conceito com respectivo conceito
    def __init__(self, cluster_model, word2vec, word2concept = None):
        self.cluster_model = cluster_model
        self.w2v = word2vec
        self.word2concept = word2concept



    #retorna menor distância entre dois conceitos, baseado no grafo de conceitos
    #concept1, concept2: conceitos para calculo da menor distância no grafo de conceitos
    #
    #returns: menor distância entre os conceitos informados
    def shortestPathDistance_concepts(self, concept1, concept2):

        #recupera distância entre os conceitos, já calculada e armazenada anteriormente
        #caso nao exista, calcula e armazena
        try:
            distance = self.conceptDistance[(concept1, concept2)]
        except KeyError:
            #distance = nx.shortest_path_length(self.conceptGraph, source=concept1, target=concept2)
            distance = nx.dijkstra_path_length(self.conceptGraph, source=concept1, target=concept2)
            #armazena distancia calculada
            self.conceptDistance[(concept1, concept2)] = distance

        return distance


    #retorna menor distância entre o conceito de duas palavras, baseado no grafo de conceitos
    #word1, word2: palavras para calculo da menor distância no grafo de conceitos
    #
    #returns: menor distância entre os conceitos informados
    def shortestPathDistance_words(self, word1, word2):

        try:
            concept1 = self.word2concept[word1]
            concept2 = self.word2concept[word2]

            distance = self.shortestPathDistance_concepts(concept1, concept2)
        except KeyError:
            #palavra não existe
            distance = -1

        #print('shortestPathDistance word|concept ', word1, concept1, word2, concept2, distance)

        return distance # self.shortestPathDistance_concepts(concept1, concept2)


    #gera grafo de relacionamento entre os conceitos
    #degree = quantidade de relacionamentos de cada conceito (grau/degree, na definição da teoria dos grafos). Default = 3
    def generateConceptGraph(self, degree = 3):

        self.conceptGraph = nx.Graph()

        #recupera centroids do cluster de conceitos
        centroids = self.cluster_model.cluster_centers_
        #qtd_clusters = len(centroids)

        #calcula as distancias entres todos os centroids
        all_distances = sdist.cdist(centroids, centroids, metric='euclidean')

        #para cada cluster , recupera as degree_th menores distancias aos demais (desprezando a distancia 0, que
        #corresponde ao próprio), e adiciona ao grafo como aresta (edge). Como o menor invariavelmente será a distancia
        #ao proprio cluster (distancia=0), o array closests_clusters_indexes terá um elemento a mais (degree+1). Este elemento
        #será desconsiderado na inclusão do edge ao grafo
        for id_cluster, cluster_distances in enumerate(all_distances):
            closests_clusters_indexes = bn.argpartition(cluster_distances, degree)[:degree+1]

            print

            #adiciona os edges entre clusters ao grafo, com 'weight' = distância entre os nodes, exceto o que corresponde ao proprio cluster
            edges = [(id_cluster, closest_cluster, {'weight': cluster_distances[closest_cluster]}) for closest_cluster in closests_clusters_indexes if closest_cluster != id_cluster]
            self.conceptGraph.add_edges_from(edges)


        return all_distances



    #atualiza vocabulário do word2vec
    #deve ser executado antes do primeiro predict() ou predictByCentroid()
    def update_vocab(self, sentences):
        self.w2v.min_count = 1
        self.w2v.build_vocab(corpus_iterable=sentences, update=True)  # update = True ensures that words are added to vocab
        self.w2v.train(corpus_iterable=sentences, epochs=iterations, total_examples=len(sentences))


    #retorna o vector de características Word2vec da palavra informada
    #word: palavra que se pretende o vetor
    #
    #returns: vetor word2vec
    def getVector(self, word):
        try:
            return self.w2v.get_vector(word, norm=True)

        except:
            #erro caso a palavra não tenha sido considerada na geração original do word2vec
            print('Palavra inexistente no vocabulário do word2vec: %s.' %word)
            return np.zeros_like(self.w2v.vectors[0])


    #retorna indice do cluster relativo a uma palavra
    #word: palavra para identificar o conceito
    #
    #returns: índice do conceito
    def predict(self, word):

        #retorna predicto do vetor (palavra) informada
        return self.cluster_model.predict([self.getVector(word)])[0]


    #retorna indice do cluster relativo a uma palavra baseado na proximidade
    #com o centroide de cada cluster
    #word: palavra para identificar o conceito
    #
    #returns: índice do conceito
    def predictByCentroid(self, word):

        #verifica conceito da palavra a partir do dicionário recebido no construtor da classe
        if (self.word2concept is not None):
            try:
                return self.word2concept[word]

            #caso a palavra nao exista no mapeamento, ignora erro e prossegue metodo para calcular
            except KeyError:
                pass


        #caso o mapeamento palavra-> conceito não houver sido informado,calcula

        #recupera vetor da palavra
        word_vector = self.getVector(word)

        #centroides do modelo (s)k-means
        centroids = self.cluster_model.cluster_centers_

        #array, de tamanho len(centroids), das distancias da palavra aos centroides
        distances = np.ones(len(centroids))

        #calcula distância da palavra a cada centroide
        for i in range(distances.size):
            distances[i] = euclidean(word_vector, centroids[i])

        #retorna indice da menor distância, o que corresponde ao cluster
        return np.argmin(distances)


    #retorna a distância de uma palavra ao centroide de um dado conceito
    #word: palavra
    #concept: conceito para calcular a distância da palavra ao seu centroid
    #
    #returns: distância da palavra ao centroid do conceito informado
    def getDistanceConceptCentroid(self, word, concept):

        #recupera vetor da palavra
        word_vector = self.getVector(word)

        #centroides do conceito informado
        centroid = self.cluster_model.cluster_centers_[concept]

        return euclidean(word_vector, centroid)



In [ ]:
# classe WeigthedVectors
# gera a vetorização ponderada por meio de clusters de conceitos e do
# word2vec do thesaurus
class WeigthedVectors:

    #cluster de conceitos (classe conceptCluster)
    concepts = None

    #word2vec do thesaurus
    w2vth = None
    index_to_key_thesaurus = None

    #dictionary com a menor distancia de uma palavra a um item do thesaurus
    word_distance_to_thesaurus = {}

    #construtor
    #concepts: cluster de conceitos (classe conceptCluster)
    #word2vec_thesaurus: word2vec do thesauros
    def __init__(self, concepts, word2vec_thesaurus, index_to_key_thesaurus = None):

        self.concepts = concepts

        self.w2vth = word2vec_thesaurus
        self.index_to_key_thesaurus = index_to_key_thesaurus

    #calcula o peso da palavra perante a lista do thesauros
    #word: palavra a ter o peso calculado
    #
    #returns peso da palavra
    def getWeightByWord(self, word):

        #caso a palavra pertença ao tesauro, retorna 0
        if word in self.index_to_key_thesaurus:
            return 0

        #palavra não pertence ao tesauro:

        #se a menor distancia da palavra a um termo do tesauro já tiver sido calculada
        #(armazenada no dictionary self.word_distance_to_thesaurus), retorna-a. Caso contrário, continua o cálculo
        try:
            return self.word_distance_to_thesaurus[word]
        except KeyError:
            pass


        #ainda não foi calculada a menor distancia do palavra:
        #calcula a distancia para cada palavra do tesauro (por meio do word2vec)
        #armazena o valor calculado em self.word_distance_to_thesaurus
        # e retorna o menor valor encontrado

        #array, de tamanho len(self.wv2vth), das distancias da palavra a cada um dos vetores do tesauro
        thesaurus_vectors = self.w2vth
        distances = np.ones(len(thesaurus_vectors))

        #recupera vetor da palavra
        word_vector = self.concepts.getVector(word)

        #calcula distância da palavra a cada termo do tesauro
        for i in range(distances.size):
            distances[i]  = euclidean(word_vector, thesaurus_vectors[i])

        #obtem a menor distancia
        #caso seja maior que 1, "normaliza", ou seja, define o valor como .99, uma vez que o peso
        # é calculado como 1 - a distância (isso está sendo feito para nar ter peso "zerado")
        menorDistancia = np.amin(distances)


        #grava menor distancia da palavra a um termo do tesauro
        self.word_distance_to_thesaurus[word] = menorDistancia


        #retorna menor distancia da palavra a um termo do tesauro
        return menorDistancia



    #gera vetores ponderados a partir dos conceitos, de cada uma das sentenças
    #sentences: sentencas que terão os vetores gerados
    #
    #returns lista de histogramas calculados a partir das matrizes ponderadas de cada sentença
    def get_weighted_vectors(self, sentences):


        #lista de palavras que existem no word2vec utilizado para gerar os clusters de conceitos
        words_in_w2v = self.concepts.w2v.index_to_key

        #array com os histogramas (H) ponderados de cada uma das sentenças
        weighted_vectors = []

        #quantidade de conceitos (# of clusters)
        k = self.concepts.cluster_model.n_clusters

        #para cada sentença, tokeniza e obtem o conceito (cluster) de cada palavra
        #em seguida, calcula a ponderação p, que é distância da palavra di à sua palavra correspondente
        #mais próxima no tesauro T (self.w2vth)
        for i, sentence in enumerate(sentences):

            print('%d sentenças processadas' % (i+1), end = '\r')

            #conceito (cluster) de cada palavra da sentença que pertença ao word2vec utilizado na geração
            #dos clusters de conceitos
            conceptByWord = [self.concepts.predictByCentroid(token) for token in sentence] # if token in words_in_w2v]

            #calcula o peso para cada palavra da sentença que pertença ao word2vec utilizado na geração
            #dos clusters de conceitos
            weightByWord = [self.getWeightByWord(token) for token in sentence] # if token in words_in_w2v]

            #matriz de ativação de cada palavra da sentença, com a dimensão n x k,
            #onde n=qtde de palavras da sentença e k = qtde de conceitos (# of clusters) (definido fora do loop)
            #matriz será inicializada com zero, pois a ativação utilizada será a hard,
            #onde apenas a posição correspondente ao conceito terá o valor 1 atribuído
            n = len(sentence)
            mx_sentence = np.zeros([n,k])

            #para cada palavra, aplica a função de ativação junto com a atribuição do
            #respectivo peso
            for word_index, concept_index in enumerate(conceptByWord):
                #recupera peso da palavra
                wordWeight  = (1 - weightByWord[word_index])
                #ativa e aplica peso, simultaneamente
                mx_sentence[word_index,concept_index] = wordWeight

            #consolidação do histograma ponderado dos conceitos da sentença
            temp = np.sum(mx_sentence, axis=0)
            weighted_vectors.append(temp)


        return weighted_vectors




## Acórdão

In [ ]:
#recupera acordaos
acordaos = pd.read_excel(files_path + 'ground_truth.xlsx')

In [ ]:
#preprocessa acordaos
acordaos['sumariop'] = acordaos.sumario.map(preprocess)
colunas = ['sumario','sumariop']
acordaos = acordaos[colunas]
acordaos.sumario = acordaos.sumario.apply(lambda x: x.lower().split())
corpusAcordao = acordaos.sumariop

## Ground Truth

In [ ]:
#recupera ground truth
ac_gt = pd.read_excel(files_path + 'ground_truth.xlsx')

### Enunciados do ground truth

In [ ]:
#monta lista com enuciados do ground truth
dados = ac_gt.to_numpy()
enunciados_gt = []

#percorre ground truth, adicionado cada enunciado associado a um acordão a uma lista
for index_acordao in ac_gt.index:
    for index_enunciado in dados[index_acordao,3:]:
        if (index_enunciado == 0):
            break

        enunciados_gt.append(index_enunciado)

## Jurisprudência

In [ ]:
#recupera jurisprudencia
juris_path = files_path + 'jurisvce.txt'

#arquivo texto está no encoding ANSI (para windows)
jurisOriginal = pd.read_table(juris_path, encoding = 'ISO-8859-1')

In [ ]:
%%time
jurisOriginal['enunciadop'] = jurisOriginal.enunciado.map(preprocess)

In [ ]:
corpusJuris = jurisOriginal.enunciadop

**Seleção de amostras dos enunciados de jurisprudência**

In [ ]:
#Seleciona as amostras, por meio de Amostragem Aleatória Simples (AAS) de enunciados de jurisprudência que serão utilizadas
#na avaliação das técnicas.
#A seleção deve ser feita após a atribuição do corpus, pois este deve ser integralmente utilizado para a vetorização

import random

QTD_AMOSTRAS = 385 # valor obtido com AAS
QTD_AMOSTRAS_A_SELECIONAR = QTD_AMOSTRAS # - len(enunciados_gt)

#retorna dataframe com amostra de enunciados de jurisprudência
def seleciona_amostras_jurisprudencia():
  lista_amostras = random.sample(range(0, len(jurisOriginal)), QTD_AMOSTRAS_A_SELECIONAR)
  lista_amostras_a_selecionar = lista_amostras + enunciados_gt

  jurisAmostra = jurisOriginal.copy() #.iloc[lista_amostras_a_selecionar]
  jurisAmostra.to_excel(files_path + 'jurisamostra.xlsx', index=True)

  return jurisAmostra


juris = seleciona_amostras_jurisprudencia()

In [ ]:
# classe TimeCounter
# registro do tempo de execução
class TimeCounter:

    #lista de registros de tempos
    times = None

    #construtor
    def __init__(self):
        self.times = {}

    #inicia contador de tempo
    #
    #name: nome do contador
    def start(self, name):
        t = time.time()
        self.times[name] = {'start': t}
    #end start()


    #finaliza contador de tempo
    #
    #name: nome do contador
    def stop(self, name):
        t = time.time()
        self.times[name]['end'] = t
    #end stop()


    #imprime tempo total, em segundos, dos contadores
    #
    #name: nome do contador
    def print(self, name):
        print(name, ':', (self.getTime(name)))

    #end print()


    #retorna tempo de um contador (em segundos)
    #
    #name: nome do contador
    def getTime(self, name):
        return (self.times[name]['end'] - self.times[name]['start'])


    #imprime todas os registros
    #
    def printAll(self):
        for i in self.times.keys():
            self.print(i)
    #end printAll()

## Cálculo de similaridade entre acórdãos selecionados e a base de jurisprudência
### BoC-Thesaurus

In [ ]:
from rank_bm25 import BM25Okapi
from sklearn.preprocessing import minmax_scale



#cria matriz esparsa com valores bm25 a partir de matriz tfidf
#tfidf_vector: vetor tfidf original (csr_matrix)
#tfidf_vocab: vocabulário utilizado para a geração do tfidf
#bm25: valores bm25 (BM25Okapi)
#
#returns bm25 convertido em matriz esparsa na dimensão do tfidf original (csr_matrix)
def create_sparse_bm25(tfidf_vector, tfidf_vocab, bm25):

    #numero de docs do tfidf
    num_docs = tfidf_vector.shape[0]

    #arrays para construção de csr_matrix do tfidf redimensionado
    #sera construido a partir do formato de matriz COO: csr_array((data, (row_ind, col_ind)), [shape=(M, N)])
    rows = [] # corresponde ao idx do documento
    cols = [] # corresponde ao idx da palavra na matriz tfidf original
    data = [] # valor do bm25

    #recupera os valores tfidf das palavras de cada documento do vetor tfidf
    for idx_doc in range(num_docs):
        print(idx_doc, end = '\r')

        #doc (documento) corresponde a cada linha do vetor tfidf recuperado no formato de matriz de coordenadas,
        #que retorna apenas as coordenadas com valores (palavras do vocabulario presentes neste documento),
        #uma vez que há muita esparsidade na matriz tfidf original
        doc = tfidf_vector[idx_doc].tocoo()

        #recupera, do documento, os indices das palavras no vocabulario tfidf
        #como esta sendo usado o formato de matriz em formato de coordenada, o atributo .col retorna as palavras
        #do vocabulario tfidf existentes neste documento
        words = doc.col

        #para cada palavra do documento, recupera seu valor bm25
        for idx_word in words:
            word = tfidf_vocab[idx_word]
            bm25_value = bm25.idf[word]

            #cria coordenada, indexando por [idx_doc, idx_word] = bm25_value
            try:
                rows.append(int(idx_doc))
                cols.append(int(idx_word))
                data.append(bm25_value)
            except:
                print('****** ERRO ******')
                print(idx_doc, word, bm25_value)
                raise

        # end for idx_word in words...

    #end for idx_doc in range(num_docs)...

    return csr_matrix((data, (rows, cols)), shape=tfidf_vector.shape)




#reduz a dimensionalidade do vertor TF-IDF a partir dos conceitos obtidos n BOC-th
#tfidf_vector: vetor tfidf original (csr_matrix)
#tfidf_vocab: vocabulário utilizado para a geração do tfidf. É o mesmo da clusterização BOC-th
#num_concepts: número de conceitos
#word2concept: Dictionary com o mapeamento word -> concept
#
#returns tfidf redimensionado para shape(num_docs, num_concepts) (csr_matrix)
def redim_tfidf_by_concept(tfidf_vector, tfidf_vocab, num_concepts, word2concept):

    #numero de docs do tfidf
    num_docs = tfidf_vector.shape[0]

    #arrays para construção de csr_matrix do tfidf redimensionado
    #sera construido a partir do formato de matriz COO: csr_array((data, (row_ind, col_ind)), [shape=(M, N)])
    rows = [] # corresponde ao idx do documento
    cols = [] # corresponde ao idx do conceito da palavra no vocabulario tfidf
    data = [] # valor do tfidf. na criaçao da csr_matriz, os indices duplicados terão os valores somados, que é a intenção

    #recupera os valores tfidf das palavras de cada documento do vetor tfidf
    for idx_doc in range(num_docs):
        print(idx_doc, end = '\r')

        #doc (documento) corresponde a cada linha do vetor tfidf recuperado no formato de matriz de coordenadas,
        #que retorna apenas as coordenadas com valores (palavras do vocabulario presentes neste documento),
        #uma vez que há muita esparsidade na matriz tfidf original
        doc = tfidf_vector[idx_doc].tocoo()

        #recupera, do documento, os indices das palavras no vocabulario  tfidf
        #como esta sendo usado o formato de matriz em formato de coordenada, o atributo .col retorna as palavras
        #do vocabulario tfidf existentes neste documento
        words = doc.col

        #para cada palavra do documento, recupera seu conceito a partir da lista de features do vetor
        #tfidf e do mapeamento palavraXconceito obtido do BOC_model
        for idx_word in words:
            word = tfidf_vocab[idx_word]
            concept = word2concept.get(word)
            tfidf = tfidf_vector[idx_doc, idx_word]

            #cria coordenada, indexando por [idx_doc, concept] = tfidf
            try:
                rows.append(int(idx_doc))
                cols.append(int(concept))
                data.append(tfidf)
            except:
                print('****** ERRO ******')
                print(idx_doc, word, concept, tfidf)
                raise

        # end for idx_word in words...

    #end for idx_doc in range(num_docs)...

    return csr_matrix((data, (rows, cols)), shape=(num_docs, num_concepts))


def bocthSimilarity(arg1, arg2):

    #copia arrays para poder manipula-los sem alterar o original (numpy.arrays são objetos mutaveis)
    x = np.copy(arg1)
    y = np.copy(arg2)

    #"zera" elementos onde seu equivalente no outro vetor é igual a zero
    #desta forma, a diferença elementWise dessas coordenadas será zero

    x[np.where(y == 0)]  = 0
    y[np.where(x == 0)]  = 0

    #calcula similaridade
    return cosineSimilarity(x ,y)


#Cálculo de similaridade entre acordaos e jurisprudencia usando tecnica
#BoC-Thesaurus (os vetores de caracteristicas consideram a frequência ponderada de conceitos)
#
#boc_model: modelo BOCModel2 utilizado
#acordaos: Dataframe dos acordãos
#juris: Dataframe da amostra de Jurisprudência
#jurisOriginal: Dataframe da Jurisprudência original (completa)
#wvec_acordaos: vetores w2v BOC-Thesaurus de acórdão
#wvec_juris: vetores w2v BOC-Thesaurus de jurisprudência
#vectorsTfIdf: vetorização tf-idf do corpus. Caso não seja informada, será calculada
#vectorsTfIdf_bocth: redução da dimensionalidade a partir do BoC. Caso não seja informada, será calculada
#
#return dataframe com o resultados do cálculo das similaridades entre acordãos e jurisprudência
def similarityBocThesaurosFull(boc_model, acordaos, juris, jurisOriginal, wvec_acordaos, wvec_juris, vectorsTfIdf=None, vectorsTfIdf_bocth=None):

    #armazena o tempo de processamento de cada técnica
    tbow = []
    tboc = []
    tbocth = []
    ttfidf = []
    tbm25 = []
    ttfidfbocth = []

    #dicionarios para armazenar os enunciados recuperados e calculados
    enunciados_bow = {}
    enunciados_tfidf = {}
    enunciados_boc = {}
    enunciados_bocth = {}
    enunciados_tfidfbocth = {}

    #classificacao BM25
    t = time.time()
    print('Gerando classificação BM25...', end = '\r')
    #classificador bm25 (parametro = dataseries corpusJuris transformado em list)
    bm25 = BM25Okapi(jurisOriginal.enunciadop.tolist() + acordaos.sumariop.tolist())
    print('Gerando classificação BM25... Concluído.', time.time()-t)


    #vetorizacao tfidf caso não tenha recebido o parametro vectorsTfIdf
    if (vectorsTfIdf is None):
      #vetorizacao TF-IDF considerando todo o corpus (jurisprudencia + acordaos)
      t = time.time()
      print('Gerando vetorização TF-IDF...', end = '\r')
      vectorizerTfIdf = TfidfVectorizer(lowercase=True)
      #vetorizacao com a base de jurisprudencia original para que qualquer enunciado existente na base de
      #amostras de jurisprudencia seja localizada
      vectorsTfIdf = vectorizerTfIdf.fit_transform(pd.concat([jurisOriginal.enunciadop, acordaos.sumariop], axis=0).astype(str))

      #vocabulario tfidf
      tfidf_vocab = vectorizerTfIdf.get_feature_names_out()
      print('Gerando vetorização TF-IDF... Concluído.', time.time()-t)

    #posicao de inicio dos acordaos (ground truth) da lista de vetores jurisprudencia + acordaos
    offset_acordaos = len(juris)

    #valores bm25 no formato csr_matrix
    newbm25 = create_sparse_bm25(vectorsTfIdf, tfidf_vocab, bm25)

    #redução de dimensionalidade tfidf-bocth caso não tenha recebido o parametro vectorsTfIdf_bocth
    if (vectorsTfIdf_bocth is None):
      #aplicando redução de dimensionalidade no vetor TF-IDF
      t = time.time()
      print('Redução de dimensionalidade no vetor TF-IDF a partir do BOC-Th...')

      vectorsTfIdf_bocth = redim_tfidf_by_concept(newbm25, tfidf_vocab, boc_model.num_concept, boc_model.getDictWord2Concept())

      print('Redução de dimensionalidade no vetor TF-IDF a partir do BOC-Th... Concluído.', time.time()-t)

      print('vectorsTfIdf.shape', vectorsTfIdf.shape)
      print('vectorsTfIdf_bocth.shape', vectorsTfIdf_bocth.shape)

    #construção da classifcação bm25 apenas com o corpus da amostra
    bm25 = BM25Okapi(juris.enunciadop.tolist())

    #lista para armazenar o resultado dos calculos
    resultados = []
    #lista para armazenar os scores bm25
    juris_scores_bm25 = []

    #loop para repetições
    for runs in range(1,2):

        print('Execução', runs)
        #limpa listas
        resultados.clear()
        juris_scores_bm25.clear()

        #para cada acordao, percorre toda jurisprudencia, verificando a similaridade
        for index_acordao, sumario, sumariop in acordaos.itertuples(name=None):

            #if (index_acordao == 2):
            #    break

            #vetor tfidf do acordao
            tfidf_acordao = vectorsTfIdf[offset_acordaos + index_acordao].A[0]

            #vetor tfidf_bocth do acordao
            tfidfbocth_acordao = vectorsTfIdf_bocth[offset_acordaos + index_acordao].A[0]


            #vetor w2v do acordao
            wvec_acordao = wvec_acordaos[index_acordao]
            #wvec_acordao = wvec_acordao/np.sum(wvec_acordao)

            #wcf do acordao, que será comparado com toda a jurisprudência
            wcfAcordao = boc_model.getwcf([sumariop])

            #bow do sumario (acordao)
            bow_sumariop = boc_model._get_bow([sumariop]).A[0]

            #verifica a similaridade para cada enunciado
            for index_juris, enunciado, enunciadop in juris.itertuples(name=None):


                if (index_juris % 777 == 0):
                    print('Acórdão %d - %d enunciados processados...' % (index_acordao, index_juris+1), end = '\r')

                #calcula as similaridades

                #bow
                t = time.time()
                #recupera vetor do enunciado armazenado anteriormente
                #caso nao exista, calcula e armazena
                try:
                    bow_enunciadop = enunciados_bow[index_juris]
                except KeyError:
                    bow_enunciadop = boc_model.bow[index_juris].A[0] #recupera o bow do enunciado já calculado na BOCModel2
                    enunciados_bow[index_juris] = bow_enunciadop

                similarity_bow = cosineSimilarity(bow_sumariop, bow_enunciadop)

                tbow.append(time.time()-t)



                #tfidf
                t=time.time()
                #recupera vetor do enunciado armazenado anteriormente
                #caso nao exista, calcula e armazena
                try:
                    tfidf_enunciado = enunciados_tfidf[index_juris]
                except KeyError:
                    tfidf_enunciado = vectorsTfIdf[index_juris].A[0]
                    enunciados_tfidf[index_juris] = tfidf_enunciado

                similarity_tfidf = cosineSimilarity(tfidf_acordao, tfidf_enunciado)
                ttfidf.append(time.time()-t)


                #boc
                t=time.time()
                #recupera vetor do enunciado armazenado anteriormente
                #caso nao exista, calcula e armazena
                try:
                    wcfEnunciado = enunciados_boc[index_juris]
                except KeyError:
                    wcfEnunciado = boc_model.getwcfdoc(index_juris)
                    enunciados_boc[index_juris] = wcfEnunciado

                similarity_boc = cosineSimilarity(wcfAcordao, wcfEnunciado)
                tboc.append(time.time()-t)

                #bocth
                t=time.time()
                #recupera vetor do enunciado armazenado anteriormente
                #caso nao exista, calcula e armazena
                try:
                    wvec_enunciado = enunciados_bocth[index_juris]
                except KeyError:
                    wvec_enunciado = wvec_juris[index_juris]
                    enunciados_bocth[index_juris] = wvec_enunciado

                similarity_bocth = cosineSimilarity(wvec_acordao, wvec_enunciado)

                tbocth.append(time.time()-t)



                #weigthedVectors
                #tfidf-bocth
                t=time.time()
                #recupera vetor do enunciado armazenado anteriormente
                #caso nao exista, calcula e armazena
                try:
                    tfidfbocth_enunciado = enunciados_tfidfbocth[index_juris]
                except KeyError:
                    tfidfbocth_enunciado = vectorsTfIdf_bocth[index_juris].A[0]
                    enunciados_tfidfbocth[index_juris] = tfidfbocth_enunciado

                similarity_tfidfbocth = cosineSimilarity(tfidfbocth_acordao, tfidfbocth_enunciado)
                ttfidfbocth.append(time.time()-t)


                #registra resultado
                #linha abaixo comentada para 'economizar' uso de memoria
                resultados.append([index_acordao, index_acordao, index_juris, index_juris,
                                   similarity_bow, similarity_boc, similarity_bocth, similarity_tfidf, similarity_tfidfbocth])

            #end for index_juris...

            #bm25
            t=time.time()
            #calcula scores bm25 da query (sumariop) perante o corpus (jurisprudencia) e adiciona na lista
            juris_scores_bm25.append(bm25.get_scores(sumariop))
            tbm25.append(time.time()-t)


            print('Acórdão %d - %d enunciados processados          ' % (index_acordao, index_juris+1), end = '\r')

        #end for index_acordao...

        print()
        print('Convertendo  array de scores bm25...', end = '\r')
        t=time.time()
        #converte o array de scores para um array unidimensional com tamanho = #acordaos x #jurisprudencia
        juris_scores_bm25 = [np.concatenate(juris_scores_bm25).tolist()]
        juris_scores_bm25 = np.array(juris_scores_bm25).T
        #aplica minmax-normalization nos scores para deixá-los na faixa [0,1]
        juris_scores_bm25 = juris_scores_bm25/np.max(juris_scores_bm25)

        #concatena aos resultados das mediçoes de cada metrica os scores bm25 (está separado pois o cálculo dos scores
        #é feito de uma unica vez para cada acordao). Essa concatenação é feita adicionando uma coluna ao final
        #do array 'resultados'. Por isso é necessário fazer o transpose (.T) do array juris_scores_bm25
        resultados = np.concatenate((resultados, juris_scores_bm25), axis=1)
        print('Convertendo  array de scores bm25... Concluído.', time.time()-t)


    #end for runs...

    print('..................................................')
    print('Quantidade de execuções:', runs)
    print('Tempo médio bow  :', np.mean(tbow), '±', np.std(tbow))
    print('Tempo médio boc  :', np.mean(tboc), '±', np.std(tboc))
    print('Tempo médio bocth:', np.mean(tbocth), '±', np.std(tbocth))
    print('Tempo médio tfidf:',  np.mean(ttfidf), '±', np.std(ttfidf))
    print('Tempo médio bm25 :',  np.mean(tbm25)/len(juris), '±', np.std(tbm25))
    print('BM25 len', len(tbm25), '   sum', np.sum(tbm25))
    print('Tempo médio tfidf-bocth :',  np.mean(ttfidfbocth), '±', np.std(ttfidfbocth))
    print('..................................................')

    return resultados


### Avaliação a partir do ground truth

In [ ]:
#calculo do mAP a partir do ground truth


def calcula_mAPFull(similaridades):

    #groundTruth convertido em numpy.array
    dados = ac_gt.to_numpy()

    mapa = []
    #lista de mAP bow e boc de cada  acordao
    mAP = []
    #para cada acordao, localiza a posicao de seus enunciados associados
    #nas similaridades calculadas
    for index_acordao in ac_gt.index:
        mapa.append([index_acordao])

        #filtra as similaridades por acordao e ordena por cada técnica de cálculo de similaridade
        #a ordenacao bm25 é descendente, pois o valor é o score de similaridade do algoritmo OkapiBM25
        #a ordenacao cgs é descendente, pois o valor é a similaridade
        #os demais também, pois são relativos à cosine similarity [0,1]
        sim_ac_bow = similaridades[similaridades.idx_sumario.eq(index_acordao)].sort_values(['bow'], ascending=[False], ignore_index=True)
        sim_ac_boc = similaridades[similaridades.idx_sumario.eq(index_acordao)].sort_values(['boc'], ascending=[False], ignore_index=True)
        sim_ac_bocth = similaridades[similaridades.idx_sumario.eq(index_acordao)].sort_values(['bocth'], ascending=[False], ignore_index=True)
        sim_ac_tfidf = similaridades[similaridades.idx_sumario.eq(index_acordao)].sort_values(['tfidf'], ascending=[False], ignore_index=True)
        sim_ac_tfidfbocth = similaridades[similaridades.idx_sumario.eq(index_acordao)].sort_values(['tfidfbocth'], ascending=[False], ignore_index=True)
        sim_ac_bm25 = similaridades[similaridades.idx_sumario.eq(index_acordao)].sort_values(['bm25'], ascending=[False], ignore_index=True)

        #verifica posicao da similaridade para cada enunciado
        #dados[index_acordao,3:] = enunciados associados ao acordao (a partir da 4ª coluna)
        for index_enunciado in dados[index_acordao,3:]:

            if (index_enunciado == 0):
                break

            mapa[-1].append([index_enunciado])

            #localiza posicao do enunciado nas similaridades do acordao
            pos_bow = sim_ac_bow[sim_ac_bow.idx_enunciado.eq(index_enunciado)].index.values[0]
            pos_boc = sim_ac_boc[sim_ac_boc.idx_enunciado.eq(index_enunciado)].index.values[0]
            pos_bocth = sim_ac_bocth[sim_ac_bocth.idx_enunciado.eq(index_enunciado)].index.values[0]
            pos_tfidf = sim_ac_tfidf[sim_ac_tfidf.idx_enunciado.eq(index_enunciado)].index.values[0]
            pos_tfidfbocth = sim_ac_tfidfbocth[sim_ac_tfidfbocth.idx_enunciado.eq(index_enunciado)].index.values[0]
            pos_bm25 = sim_ac_bm25[sim_ac_bm25.idx_enunciado.eq(index_enunciado)].index.values[0]

            mapa[-1][-1].append(['bow',pos_bow])
            mapa[-1][-1].append(['boc',pos_boc])
            mapa[-1][-1].append(['bocth',pos_bocth])
            mapa[-1][-1].append(['tfidf',pos_tfidf])
            mapa[-1][-1].append(['tfidfbocth',pos_tfidfbocth])
            mapa[-1][-1].append(['bm25',pos_bm25])

        #end for index_enunciado...

        #cálculo do mAP
        #bow
        ordered_Bow = sorted(mapa[-1][1:], key=lambda bow: bow[1][1])
        arr_mAPBow = []
        for i in range(len(ordered_Bow)):
            valor = (i+1)/(ordered_Bow[i][1][1]+1)
            arr_mAPBow.append(valor)

        mAPBow = np.mean(arr_mAPBow)
        stdBow  = np.std(arr_mAPBow)


        #boc
        ordered_Boc = sorted(mapa[-1][1:], key=lambda boc: boc[2][1])
        arr_mAPBoc = []
        for i in range(len(ordered_Boc)):
            valor = (i+1)/(ordered_Boc[i][2][1]+1)
            arr_mAPBoc.append(valor)

        mAPBoc = np.mean(arr_mAPBoc)
        stdBoc  = np.std(arr_mAPBoc)


        #bocth
        ordered_Bocth = sorted(mapa[-1][1:], key=lambda bocth: bocth[3][1])
        arr_mAPBocth = []
        for i in range(len(ordered_Bocth)):
            valor = (i+1)/(ordered_Bocth[i][3][1]+1)
            arr_mAPBocth.append(valor)

        mAPBocth = np.mean(arr_mAPBocth)
        stdBocth  = np.std(arr_mAPBocth)

        #tfidf
        ordered_Tfidf = sorted(mapa[-1][1:], key=lambda tfidf: tfidf[4][1])
        arr_mAPTfidf = []
        for i in range(len(ordered_Tfidf)):
            valor = (i+1)/(ordered_Tfidf[i][4][1]+1)
            arr_mAPTfidf.append(valor)

        mAPTfidf = np.mean(arr_mAPTfidf)
        stdTfidf  = np.std(arr_mAPTfidf)


        #tfidfbocth
        ordered_Tfidfbocth = sorted(mapa[-1][1:], key=lambda tfidfbocth: tfidfbocth[5][1])
        arr_mAPTfidfbocth = []
        for i in range(len(ordered_Tfidfbocth)):
            valor = (i+1)/(ordered_Tfidfbocth[i][5][1]+1)
            arr_mAPTfidfbocth.append(valor)

        mAPTfidfbocth = np.mean(arr_mAPTfidfbocth)
        stdTfidfbocth = np.std(arr_mAPTfidfbocth)


        #bm25
        ordered_Bm25 = sorted(mapa[-1][1:], key=lambda bm25: bm25[6][1])
        arr_mAPBm25 = []
        for i in range(len(ordered_Bm25)):
            valor = (i+1)/(ordered_Bm25[i][6][1]+1)
            arr_mAPBm25.append(valor)

        mAPBm25 = np.mean(arr_mAPBm25)
        stdBm25  = np.std(arr_mAPBm25)


        #adiciona mAPs na lista de resultado
        mAP.append([index_acordao, mAPBow, mAPBoc, mAPBocth, mAPTfidf, mAPTfidfbocth, mAPBm25])

    #end for index_acordao...

    #retorna dataframe
    return DataFrame(data = mAP, columns=['idx_acordao', 'map_bow', 'map_boc', 'map_bocth', 'map_tfidf', 'map_tfidfbocth', 'map_bm25'])



In [ ]:
#calculo do precision@ a partir do ground truth

#Calcula o recall@k das similaridades
#O recall em k é a proporção de itens relevantes encontrados entres os top-k listados. Neste caso, relevante é o
#enunciado que pertencem ao ground truth de determinado acórdão. Fórmula:
#Recall@k = (# of recommended items @k that are relevant) / (total # of relevant items)
#
#k: posição objetivo na lista de similaridades para o cálculo
#
#returns
def calcula_recallAtK(similaridades, k = 10):

    #calcula o recall@k de um acordao especifico
    #listagt: lista com as posições de cada técnica, na lista de similaridades,  dos enunciados do ground truth de determinado acórdão
    #k: posição objetivo na lista de similaridades para o cálculo
    #pos: posição, em cada item da lista 'listagt', da técnica que está se calculando o recall@k
    def recallAtK(listagt, k, pos):

      #a quantidade de itens relevantes corresponde ao tamanho de itens do ground truth
      total_relevants =  len(listagt)
      #contador da quantidade de itens relavantes em k
      qty_relevants_at_k = 0

      #percorre enunciados do gt, verificando a posição do cálculo de similaridade de determinada técnica (pos) na lista de similaridades
      for item in listagt:
        #posicao do cálculo de determinada técnica lista de similaridades (indicada pelo parâmetro 'pos'). Soma + 1 pois o índice começa em 0
        position = item[pos][1]+1
        #caso a posição na lista de similaridades esteja dentro do top-k, incrementa o contador
        if position <= k:
          qty_relevants_at_k = qty_relevants_at_k + 1

      #recall@k
      return qty_relevants_at_k/total_relevants


    #groundTruth convertido em numpy.array
    dados = ac_gt.to_numpy()

    mapa = []
    #lista de mAP bow e boc de cada  acordao
    mRecallAtK = []
    #para cada acordao, localiza a posicao de seus enunciados associados
    #nas similaridades calculadas
    for index_acordao in ac_gt.index:
        mapa.append([index_acordao])

        #filtra as similaridades por acordao e ordena por cada técnica de cálculo de similaridade
        #a ordenacao bm25 é descendente, pois o valor é o score de similaridade do algoritmo OkapiBM25
        #os demais também, pois são relativos à cosine similarity [0,1]
        sim_ac_bow = similaridades[similaridades.idx_sumario.eq(index_acordao)].sort_values(['bow'], ascending=[False], ignore_index=True)
        sim_ac_boc = similaridades[similaridades.idx_sumario.eq(index_acordao)].sort_values(['boc'], ascending=[False], ignore_index=True)
        sim_ac_bocth = similaridades[similaridades.idx_sumario.eq(index_acordao)].sort_values(['bocth'], ascending=[False], ignore_index=True)
        sim_ac_tfidf = similaridades[similaridades.idx_sumario.eq(index_acordao)].sort_values(['tfidf'], ascending=[False], ignore_index=True)
        sim_ac_tfidfbocth = similaridades[similaridades.idx_sumario.eq(index_acordao)].sort_values(['tfidfbocth'], ascending=[False], ignore_index=True)
        sim_ac_bm25 = similaridades[similaridades.idx_sumario.eq(index_acordao)].sort_values(['bm25'], ascending=[False], ignore_index=True)

        #verifica posicao da similaridade para cada enunciado
        #dados[index_acordao,3:] = enunciados associados ao acordao (a partir da 4ª coluna)
        for index_enunciado in dados[index_acordao,3:]:

            if (index_enunciado == 0):
                break

            mapa[-1].append([index_enunciado])

            #localiza posicao do enunciado nas similaridades do acordao
            pos_bow = sim_ac_bow[sim_ac_bow.idx_enunciado.eq(index_enunciado)].index.values[0]
            pos_boc = sim_ac_boc[sim_ac_boc.idx_enunciado.eq(index_enunciado)].index.values[0]
            pos_bocth = sim_ac_bocth[sim_ac_bocth.idx_enunciado.eq(index_enunciado)].index.values[0]
            pos_tfidf = sim_ac_tfidf[sim_ac_tfidf.idx_enunciado.eq(index_enunciado)].index.values[0]
            pos_tfidfbocth = sim_ac_tfidfbocth[sim_ac_tfidfbocth.idx_enunciado.eq(index_enunciado)].index.values[0]
            pos_bm25 = sim_ac_bm25[sim_ac_bm25.idx_enunciado.eq(index_enunciado)].index.values[0]

            mapa[-1][-1].append(['bow',pos_bow])
            mapa[-1][-1].append(['boc',pos_boc])
            mapa[-1][-1].append(['bocth',pos_bocth])
            mapa[-1][-1].append(['tfidf',pos_tfidf])
            mapa[-1][-1].append(['tfidfbocth',pos_tfidfbocth])
            mapa[-1][-1].append(['bm25',pos_bm25])

        #end for index_enunciado...

        #cálculo do mAP
        #bow
        recallatk_bow = recallAtK(mapa[-1][1:], k, 1)

        #boc
        recallatk_boc = recallAtK(mapa[-1][1:], k, 2)

        #bocth
        recallatk_bocth = recallAtK(mapa[-1][1:], k, 3)

        #tfidf
        recallatk_tfidf = recallAtK(mapa[-1][1:], k, 4)

        #tfidfbocth
        recallatk_tfidfbocth = recallAtK(mapa[-1][1:], k, 5)

        #bm25
        recallatk_bm25 = recallAtK(mapa[-1][1:], k, 6)

        #adiciona mAPs na lista de resultado
        mRecallAtK.append([index_acordao, recallatk_bow, recallatk_boc, recallatk_bocth, recallatk_tfidf, recallatk_tfidfbocth, recallatk_bm25])

    #end for index_acordao...


    #retorna dataframe
    return DataFrame(data = mRecallAtK, columns=['idx_acordao', 'recallatk_bow', 'recallatk_boc', 'recallatk_bocth', 'recallatk_tfidf', 'recallatk_tfidfbocth', 'recallatk_bm25'])


In [ ]:
#geracao de word2vec do thesaurus a partir do w2v do corpus
def w2vThesaurusByCorpus(thesaurus_words, w2vCorpus):
    #lista temporaria
    thw2v = []
    #percorre palavras do thesaurus, recuperando o w2v de cada uma delas
    for i in range(len(thesaurus_words)):
        try:
            #recupera indice do palavra do thesaurus nos vetores do w2v do corpus
            index = w2vCorpus.index_to_key.index(thesaurus_words[i])
            #adiciona w2v da palavra do thesaurus na lista temporaria
            thw2v.append(w2vCorpus.vectors[index])
        except ValueError:
            #caso a palavra do thesaurus nao esteja presente no corpus, o indice nao sera localizado e ocorrera um erro
            #assim, sera adicionado um vetor de zeros, com a mesma dimensao dos vetores do w2v
            thw2v.append(np.zeros_like(w2vCorpus.vectors[0]))

    #retorna numpy.array
    return np.asarray(thw2v)


In [ ]:
#geracao de word2vec a partir de outro word2vec
#vocab: lista de palavras para obtenção dos respectivos embeddings em w2v
#w2v: word2vec com os embeddings a serem recuperados para cada palavra em vocab
#
#returns vetor word2vec
def generateW2vByVocab(vocab, w2v):
    #lista temporaria
    w2v_temp = []
    #percorre palavras do vocabulario, recuperando o w2v de cada uma delas
    for word in vocab:
        try:
            #adiciona w2v da palavra na lista temporaria
            w2v_temp.append(w2v[word])
        except KeyError:
            #caso a palavra não exista em w2v, o indice nao sera localizado e ocorrera um erro
            #assim, sera adicionado um vetor de zeros, com a mesma dimensao dos vetores do w2v
            w2v_temp.append(np.zeros_like(w2v.vectors[0]))

    #retorna numpy.array
    return np.asarray(w2v_temp)


### Execução dos testes

In [ ]:
%%time

tc = TimeCounter()

#configuraçao default para word2vec
embedding_dim = 300
context = 5
min_freq = 1
iterations = 5
concepts = 300

#quantidade de execuções por setup
NUMBER_OF_EXECUTIONS = 1


boc_model = None

runningList = []

runningList.append({'desc':'D300C300-SKM' ,'embedding_dim':300, 'concepts': 300, 'clustermethod': 'skm', 'normalize': True})
runningList.append({'desc':'D300C300-KM' ,'embedding_dim':300, 'concepts': 300, 'clustermethod': 'km', 'normalize': True})

runningList.append({'desc':'D400C300-SKM' ,'embedding_dim':400, 'concepts': 300, 'clustermethod': 'skm', 'normalize': True})
runningList.append({'desc':'D400C300-KM' ,'embedding_dim':400, 'concepts': 300, 'clustermethod': 'km', 'normalize': True})

runningList.append({'desc':'D500C300-SKM' ,'embedding_dim':500, 'concepts': 300, 'clustermethod': 'skm', 'normalize': True})
runningList.append({'desc':'D500C300-KM' ,'embedding_dim':500, 'concepts': 300, 'clustermethod': 'km', 'normalize': True})

runningList.append({'desc':'D300C400-SKM' ,'embedding_dim':300, 'concepts': 400, 'clustermethod': 'skm', 'normalize': True})
runningList.append({'desc':'D300C400-KM' ,'embedding_dim':300, 'concepts': 400, 'clustermethod': 'km', 'normalize': True})

runningList.append({'desc':'D400C400-SKM' ,'embedding_dim':400, 'concepts': 400, 'clustermethod': 'skm', 'normalize': True})
runningList.append({'desc':'D400C400-KM' ,'embedding_dim':400, 'concepts': 400, 'clustermethod': 'km', 'normalize': True})

runningList.append({'desc':'D500C400-SKM' ,'embedding_dim':500, 'concepts': 400, 'clustermethod': 'skm', 'normalize': True})
runningList.append({'desc':'D500C400-KM' ,'embedding_dim':500, 'concepts': 400, 'clustermethod': 'km', 'normalize': True})

#list de dictionary com o resultado das execuções
resultados = []

#vetorizacao TF-IDF considerando todo o corpus (jurisprudencia + acordaos)
#calculado fora do loop pois é a mesma vetorização sempre, independentemente da dimensão do word2vec e dos conceitos
t = time.time()
print('Gerando vetorização TF-IDF *...', end = '\r')
vectorizerTfIdf = TfidfVectorizer(lowercase=True)
#vetorizacao com a base de jurisprudencia original para que qualquer enunciado existente na base de
#amostras de jurisprudencia seja localizada
vectorsTfIdf = vectorizerTfIdf.fit_transform(pd.concat([jurisOriginal.enunciadop, acordaos.sumariop], axis=0).astype(str))
print('Gerando vetorização TF-IDF *... Concluído.', time.time()-t)

#gera amostra de jusrisprudência, que será utilizada no cálculo de similaridade para todos os setup,
#exceto se NUMBER_OF_EXECUTIONS for maior que 1 (verificação feita mais abaixo, dentro do loop de execuções)
juris = seleciona_amostras_jurisprudencia()

for params in runningList:

    print()
    print(params['desc'])
    tc.start(params['desc'])

    #gera w2v da jurisprudencia (e do acordao), para ter o vocabulario completo
    print('Gerando word2vec da Jurisprudência + acórdãos...')
    tc.start('w2v juris')
    wv_juris = Word2Vec(sentences=list(corpusJuris)+list(corpusAcordao), vector_size=params['embedding_dim'], window=context,
                        min_count=min_freq, epochs=iterations, sg=1).wv

    #print('Gerando wv_juris a partir de glove_s300... Concluído. wv_juris.shape:', wv_juris.shape)
    tc.stop('w2v juris')
    print('Gerando word2vec da Jurisprudência + acórdãos... Concluído.' , tc.getTime('w2v juris'))

    #word2vec do VCE a partir do w2v do corpus
    tc.start('w2v thesaurus')
    print('Gerando word2vec do thesaurus a partir do corpus...')
    wv_thesaurus_corpus = generateW2vByVocab(thesaurus_words_lemma, wv_juris)
    print('wv_thesaurus_corpus.shape', wv_thesaurus_corpus.shape)
    tc.stop('w2v thesaurus')
    print('Gerando word2vec do thesaurus a partir do corpus... Concluído.', tc.getTime('w2v thesaurus'))


    #modelo bag-of-concepts
    print('Gerando BOCModel2...', end = '\r')
    tc.start('bocmodel')
    boc_model = BOCModel2(corpusJuris, wv_juris.vectors, wv_juris.index_to_key,
                          num_concept=params['concepts'], num_embedding_dim=params['embedding_dim'],
                          num_context=context, clusterMethod=params['clustermethod'], loadCluster=False)
    boc_matrix, word2concept_list, idx2word_converter = boc_model.fit()
    tc.stop('bocmodel')
    print('Gerando BOCModel2... Concluído.', tc.getTime('bocmodel'))


    #predição de conceitos
    print('Gerando conceptCluster...', end = '\r')
    concept = conceptCluster(boc_model.getClusterClass(), wv_juris, dict(word2concept_list))
    #gera grafo de conceitos
    concept.generateConceptGraph()
    print('Gerando conceptCluster... Concluído.')

    #classe para geração dos vetores ponderados
    wvectors = WeigthedVectors(concept, wv_thesaurus_corpus, thesaurus_words_lemma)

    #vetores ponderados da jurisprudencia
    print('Gerando vetores ponderados da Jurisprudência...')
    tc.start('wvecjuris')
    wvec_juris = wvectors.get_weighted_vectors(corpusJuris)
    tc.stop('wvecjuris')
    print('Gerando vetores ponderados da Jurisprudência... Concluído.', tc.getTime('wvecjuris'))

    #vetores ponderados dos acordaos
    print('Gerando vetores ponderados dos Acórdãos...')
    tc.start('wvecacordaos')
    wvec_acordaos = wvectors.get_weighted_vectors(corpusAcordao)
    tc.stop('wvecacordaos')
    print('Gerando vetores ponderados dos Acórdãos... Concluído.', tc.getTime('wvecacordaos'))

    #normaliza vetores bocth
    vtemp = np.concatenate((wvec_juris,wvec_acordaos))
    wvec_juris = wvec_juris/np.max(vtemp)
    wvec_acordaos = wvec_acordaos/np.max(vtemp)


    #mAP
    mapbowmeanfull = []
    mapbowstdfull = []
    mapbocmeanfull = []
    mapbocstdfull = []
    mapbocthmeanfull = []
    mapbocthstdfull = []
    maptfidfmeanfull = []
    maptfidfstdfull = []
    maptfidfbocthmeanfull = []
    maptfidfbocthstdfull = []
    mapbm25meanfull = []
    mapbm25stdfull = []





    #recall@k
    recallatkbowmeanfull = []
    recallatkbowstdfull = []
    recallatkbocmeanfull = []
    recallatkbocstdfull = []
    recallatkbocthmeanfull = []
    recallatkbocthstdfull = []
    recallatktfidfmeanfull = []
    recallatktfidfstdfull = []
    recallatktfidfbocthmeanfull = []
    recallatktfidfbocthstdfull = []
    recallatkbm25meanfull = []
    recallatkbm25stdfull = []


    #geração do tfidf-bocth
    #aplicando redução de dimensionalidade no vetor TF-IDF
    t = time.time()
    print('Redução de dimensionalidade no vetor TF-IDF a partir do BOC-Th *...')
    tfidf_vocab = vectorizerTfIdf.get_feature_names_out()
    vectorsTfIdf_bocth = redim_tfidf_by_concept(vectorsTfIdf, tfidf_vocab,
                                                boc_model.num_concept, boc_model.getDictWord2Concept())
    print('Redução de dimensionalidade no vetor TF-IDF a partir do BOC-Th *... Concluído.', time.time()-t)
    print('vectorsTfIdf.shape *', vectorsTfIdf.shape)
    print('vectorsTfIdf_bocth.shape *', vectorsTfIdf_bocth.shape)

    #executa NUMBER_OF_EXECUTIONS o ciclo completo de calculo de similaridades
    for execution in range(1, NUMBER_OF_EXECUTIONS+1):

      #gera nova amostra de jurisprudencia somente se o numero de execuções for maior que 1
      #caso contrário, mantém a mesma amostra para todos os setups de dimensão word2vec e conceitos
      if (NUMBER_OF_EXECUTIONS > 1):
          juris = seleciona_amostras_jurisprudencia()

      #calcula similaridade usando Boc-Th
      print()
      print('*********************************')
      print('Cálculo da similaridade', params['desc'] + '-' + str(execution) + '...')
      tc.start('similaridade')
      dados_bocTh = similarityBocThesaurosFull(boc_model, acordaos, juris, jurisOriginal, wvec_acordaos, wvec_juris) #, vectorsTfIdf, vectorsTfIdf_bocth)
      tc.stop('similaridade')
      print('Cálculo da similaridade concluído.', tc.getTime('similaridade'))

      #cria dataframe e grava resultado
      print('Geração de dataframe...', end = '\r')
      similaridades =  DataFrame(data = dados_bocTh, columns=['idx_sumario','sumario','idx_enunciado','enunciado',
                                                    'bow', 'boc', 'bocth', 'tfidf', 'tfidfbocth', 'bm25'])

      #ajuste dos tipos das colunas pois o array recebido foi concatenado com numpy.concatenate, o que transforma todos os
      #tipos de colunas em um unico, no caso, string
      convert_dict = {'idx_sumario': int,
                      'sumario': int,
                      'idx_enunciado': int,
                      'enunciado': int,
                      'bow': float,
                      'boc': float,
                      'bocth': float,
                      'tfidf': float,
                      'tfidfbocth': float,
                      'bm25': float}
      similaridades = similaridades.astype(convert_dict)

      print('Geração de dataframe... Concluído.')


      tc.start('map')
      df_map = calcula_mAPFull(similaridades)
      tc.stop('map')
      print('Cálculo do mAP concluído.', tc.getTime('map'))
      print()

      tc.start('recallAtk')
      df_recallatk = calcula_recallAtK(similaridades,100)
      tc.stop('recallAtk')
      print('Cálculo do recallAtk concluído.', tc.getTime('recallAtk'))



      #mAP
      mapbowmeanfull.append(df_map.map_bow.mean())
      mapbowstdfull.append(df_map.map_bow.std())
      mapbocmeanfull.append(df_map.map_boc.mean())
      mapbocstdfull.append(df_map.map_boc.std())
      mapbocthmeanfull.append(df_map.map_bocth.mean())
      mapbocthstdfull.append(df_map.map_bocth.std())
      maptfidfmeanfull.append(df_map.map_tfidf.mean())
      maptfidfstdfull.append(df_map.map_tfidf.std())
      maptfidfbocthmeanfull.append(df_map.map_tfidfbocth.mean())
      maptfidfbocthstdfull.append(df_map.map_tfidfbocth.std())
      mapbm25meanfull.append(df_map.map_bm25.mean())
      mapbm25stdfull.append(df_map.map_bm25.std())

      #recall@k
      recallatkbowmeanfull.append(df_recallatk.recallatk_bow.mean())
      recallatkbowstdfull.append(df_recallatk.recallatk_bow.std())
      recallatkbocmeanfull.append(df_recallatk.recallatk_boc.mean())
      recallatkbocstdfull.append(df_recallatk.recallatk_boc.std())
      recallatkbocthmeanfull.append(df_recallatk.recallatk_bocth.mean())
      recallatkbocthstdfull.append(df_recallatk.recallatk_bocth.std())
      recallatktfidfmeanfull.append(df_recallatk.recallatk_tfidf.mean())
      recallatktfidfstdfull.append(df_recallatk.recallatk_tfidf.std())
      recallatktfidfbocthmeanfull.append(df_recallatk.recallatk_tfidfbocth.mean())
      recallatktfidfbocthstdfull.append(df_recallatk.recallatk_tfidfbocth.std())
      recallatkbm25meanfull.append(df_recallatk.recallatk_bm25.mean())
      recallatkbm25stdfull.append(df_recallatk.recallatk_bm25.std())


      if (NUMBER_OF_EXECUTIONS > 1):
        print('*********************************')
        print(params['desc'] + '-' + str(execution))
        print('mAP bow:', mapbowmeanfull[-1],'±', mapbowstdfull[-1])
        print('mAP boc:', mapbocmeanfull[-1],'±', mapbocstdfull[-1])
        print('mAP bocth:', mapbocthmeanfull[-1],'±', mapbocthstdfull[-1])
        print('mAP tfidf:', maptfidfmeanfull[-1],'±', maptfidfstdfull[-1])
        print('mAP tfidfbocth:', maptfidfbocthmeanfull[-1],'±', maptfidfbocthstdfull[-1])
        print('mAP bm25:', mapbm25meanfull[-1],'±', mapbm25stdfull[-1])
        print()
        print('recall@k bow:', recallatkbowmeanfull[-1],'±', recallatkbowstdfull[-1])
        print('recall@k boc:', recallatkbocmeanfull[-1],'±', recallatkbocstdfull[-1])
        print('recall@k bocth:', recallatkbocthmeanfull[-1],'±', recallatkbocthstdfull[-1])
        print('recall@k tfidf:', recallatktfidfmeanfull[-1],'±', recallatktfidfstdfull[-1])
        print('recall@k tfidfbocth:', recallatktfidfbocthmeanfull[-1],'±', recallatktfidfbocthstdfull[-1])
        print('recall@k bm25:', recallatkbm25meanfull[-1],'±', recallatkbm25stdfull[-1])


      # end for execution...


    resultados.append({'desc': params['desc'],
                     'mapbowmean':np.mean(mapbowmeanfull), 'mapbowstd':np.mean(mapbowstdfull),
                     'mapbocmean':np.mean(mapbocmeanfull), 'mapbocstd':np.mean(mapbocstdfull),
                     'mapbocthmean':np.mean(mapbocthmeanfull), 'mapbocthstd':np.mean(mapbocthstdfull),
                     'maptfidfmean':np.mean(maptfidfmeanfull), 'maptfidfstd':np.mean(maptfidfstdfull),
                     'maptfidfbocthmean':np.mean(maptfidfbocthmeanfull), 'maptfidfbocthstd':np.mean(maptfidfbocthstdfull),
                     'mapbm25mean':np.mean(mapbm25meanfull), 'mapbm25std':np.mean(mapbm25stdfull),

                     'recallatkbowmean':np.mean(recallatkbowmeanfull), 'recallatkbowstd':np.mean(recallatkbowstdfull),
                     'recallatkbocmean':np.mean(recallatkbocmeanfull), 'recallatkbocstd':np.mean(recallatkbocstdfull),
                     'recallatkbocthmean':np.mean(recallatkbocthmeanfull), 'recallatkbocthstd':np.mean(recallatkbocthstdfull),
                     'recallatktfidfmean':np.mean(recallatktfidfmeanfull), 'recallatktfidfstd':np.mean(recallatktfidfstdfull),
                     'recallatktfidfbocthmean':np.mean(recallatktfidfbocthmeanfull), 'recallatktfidfbocthstd':np.mean(recallatktfidfbocthstdfull),
                     'recallatkbm25mean':np.mean(recallatkbm25meanfull), 'recallatkbm25std':np.mean(recallatkbm25stdfull)})

    tc.stop(params['desc'])

    print('*********************************')
    r = resultados[-1] #ultimo registro na lista 'resultados'
    print(r['desc'])
    print('mAP bow:', r['mapbowmean'],'±', r['mapbowstd'])
    print('mAP boc:', r['mapbocmean'],'±', r['mapbocstd'])
    print('mAP bocth:', r['mapbocthmean'],'±', r['mapbocthstd'])
    print('mAP tfidf:', r['maptfidfmean'],'±', r['maptfidfstd'])
    print('mAP tfidfbocth:', r['maptfidfbocthmean'],'±', r['maptfidfbocthstd'])
    print('mAP bm25:', r['mapbm25mean'],'±', r['mapbm25std'])
    print()

    print('recall@k bow:', r['recallatkbowmean'],'±', r['recallatkbowstd'])
    print('recall@k boc:', r['recallatkbocmean'],'±', r['recallatkbocstd'])
    print('recall@k bocth:', r['recallatkbocthmean'],'±', r['recallatkbocthstd'])
    print('recall@k tfidf:', r['recallatktfidfmean'],'±', r['recallatktfidfstd'])
    print('recall@k tfidfbocth:', r['recallatktfidfbocthmean'],'±', r['recallatktfidfbocthstd'])
    print('recall@k bm25:', r['recallatkbm25mean'],'±', r['recallatkbm25std'])
    print('*********************************')
    print(params['desc'])
    print('(tempo', tc.getTime(params['desc']), ')')
    print('*********************************')
    print()

#end for param...

print()
print()
for r in resultados:
    print('*********************************')
    print(r['desc'])
    print('mAP bow:', r['mapbowmean'],'±', r['mapbowstd'])
    print('mAP boc:', r['mapbocmean'],'±', r['mapbocstd'])
    print('mAP bocth:', r['mapbocthmean'],'±', r['mapbocthstd'])
    print('mAP tfidf:', r['maptfidfmean'],'±', r['maptfidfstd'])
    print('mAP tfidfbocth:', r['maptfidfbocthmean'],'±', r['maptfidfbocthstd'])
    print('mAP bm25:', r['mapbm25mean'],'±', r['mapbm25std'])
    print()
    print('recall@k bow:', r['recallatkbowmean'],'±', r['recallatkbowstd'])
    print('recall@k boc:', r['recallatkbocmean'],'±', r['recallatkbocstd'])
    print('recall@k bocth:', r['recallatkbocthmean'],'±', r['recallatkbocthstd'])
    print('recall@k tfidf:', r['recallatktfidfmean'],'±', r['recallatktfidfstd'])
    print('recall@k tfidfbocth:', r['recallatktfidfbocthmean'],'±', r['recallatktfidfbocthstd'])
    print('recall@k bm25:', r['recallatkbm25mean'],'±', r['recallatkbm25std'])




print()

pd.DataFrame(resultados).to_excel(files_path + 'resultado_similaridadeFull.xlsx', index=False)
print(files_path + 'resultado_similaridadeFull.xlsx gerado.')
